In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import statistics
import re

import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Load data
dtset = pd.read_csv("../../Hanoi-Housing-Prices-Prediction/Preprocessing/nhaban_clean.csv")
dtset_columns = dtset.columns.tolist()
print("+ Shape: ", dtset.shape) # (Number of Lines, Number of Columns)
print("+ Column: ", dtset_columns) # Name the attribute columns.
dtset.info()

+ Shape:  (28658, 20)
+ Column:  ['Tiêu đề', 'Địa chỉ', 'Ngày đăng', 'Ngày hết hạn', 'Loại tin', 'Mã tin', 'Người đăng', 'Thông tin mô tả', 'Diện tích', 'Mức giá', 'Mặt tiền', 'Đường vào', 'Hướng ban công', 'Hướng nhà', 'Số tầng', 'Số phòng ngủ', 'Số toilet', 'Pháp lý', 'Nội thất', 'Loại hình']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28658 entries, 0 to 28657
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tiêu đề          28658 non-null  object 
 1   Địa chỉ          28658 non-null  object 
 2   Ngày đăng        28658 non-null  object 
 3   Ngày hết hạn     28658 non-null  object 
 4   Loại tin         28658 non-null  object 
 5   Mã tin           28658 non-null  int64  
 6   Người đăng       28658 non-null  object 
 7   Thông tin mô tả  28658 non-null  object 
 8   Diện tích        28658 non-null  float64
 9   Mức giá          28658 non-null  object 
 10  Mặt tiền         18348 non-null  objec

### Delete attribute columns

In [3]:
# Print the oldest and newest posting dates
print("Oldest posting date:", dtset['Ngày đăng'].min())
print("Newest posting date:", dtset['Ngày đăng'].max())

# Print the oldest and newest expiration dates
print("Oldest expiration date:", dtset['Ngày hết hạn'].min())
print("Newest expiration date:", dtset['Ngày hết hạn'].max())

Oldest posting date: 2024-02-04
Newest posting date: 2024-03-15
Oldest expiration date: 2024-03-04
Newest expiration date: 2024-04-11


Notice that the gap between the posting date and the expiration date is negligible (only about 1 month) => The range of house price fluctuations in that range will not be too large => Skip this section, drop the "Ngày đăng" and "Ngày hết hạn" columns

"Người đăng" feature has no value => drop 

In [4]:
# Print the initial dataset size
print("Initial dataset size:", dtset.shape)

# Drop specified columns
dtset.drop(['Ngày đăng', 'Ngày hết hạn', 'Người đăng'], axis=1, inplace=True)

# Print the dataset size after removing some attribute columns
print("Dataset size after removing some attribute columns:", dtset.shape)

Initial dataset size: (28658, 20)
Dataset size after removing some attribute columns: (28658, 17)


### Processing the "Địa chỉ" column

In [5]:
dtset['Địa chỉ'].unique()

array(['Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dương Nội, Hà Đông, Hà Nội',
       'Dự án An Khang Villa, Đường Tố Hữu, Phường La Khê, Hà Đông, Hà Nội',
       'Dự án Hado Charm Villas, Đường Đại lộ Thăng Long, Xã An Thượng, Hoài Đức, Hà Nội.',
       ..., 'Phường Hàng Bạc, Hoàn Kiếm, Hà Nội',
       'Đường Lương Ngọc Quyến, Phường Hàng Buồm, Hoàn Kiếm, Hà Nội',
       'Phố Hàng Dầu, Phường Hàng Bạc, Hoàn Kiếm, Hà Nội.'], dtype=object)

In [6]:
# Loop through the dataset and print full address information
for address in dtset['Địa chỉ']:
    print(address)

Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dương Nội, Hà Đông, Hà Nội
Dự án An Khang Villa, Đường Tố Hữu, Phường La Khê, Hà Đông, Hà Nội
Dự án Hado Charm Villas, Đường Đại lộ Thăng Long, Xã An Thượng, Hoài Đức, Hà Nội.
Phố Cầu Cốc, Phường Tây Mỗ, Nam Từ Liêm, Hà Nội.
Dự án 671 Hoàng Hoa Thám, Đường Hoàng Hoa Thám, Phường Liễu Giai, Ba Đình, Hà Nội
Đường Xuân Đỉnh, Phường Xuân Đỉnh, Bắc Từ Liêm, Hà Nội
Đường Đình Thôn, Phường Mỹ Đình 1, Nam Từ Liêm, Hà Nội
Phố Từ Hoa, Phường Quảng An, Tây Hồ, Hà Nội
Phố Đại La, Phường Đồng Tâm, Hai Bà Trưng, Hà Nội
Phố Khâm Thiên, Phường Khâm Thiên, Đống Đa, Hà Nội
Dự án Hinode Royal Park, Đường Quốc lộ 32, Xã Di Trạch, Hoài Đức, Hà Nội
Phố Trần Cung, Phường Cổ Nhuế 1, Bắc Từ Liêm, Hà Nội
Phố Trung Kính, Phường Yên Hòa, Cầu Giấy, Hà Nội
Đường Trịnh Văn Bô, Phường Xuân Phương, Nam Từ Liêm, Hà Nội.
Dự án Roman Plaza, Đường Tố Hữu, Phường Đại Mỗ, Nam Từ Liêm, Hà Nội.
Đường Phương Canh, Phường Phương Canh, Nam Từ Liêm, Hà Nội.
Đường Đức Diễn, Phường Phú D

Add "Đường/Phố" column

In [7]:
# Create a list to store street information
streets = [np.nan for i in range(dtset.shape[0])]
print("Streets length before =", len(streets))  # "streets" is a new column named "Đường" extracted from the "Khu vực" column.
print("Streets sample before:", streets[0:3])
street_exception_list = []  # Contains exceptional cases in a list variable.

Streets length before = 28658
Streets sample before: [nan, nan, nan]


In [8]:
# Create a list to store street information
streets = []

# Loop through the dataset
for index in range(dtset.shape[0]):
    # Skip if the value of 'Địa chỉ' at index 'index' is NaN
    if pd.isnull(dtset['Địa chỉ'][index]):
        streets.append(np.nan)  # Append NaN if address is NaN
        continue
    
    # Split address by ", " to extract components
    address_components = dtset['Địa chỉ'][index].split(", ")

    street_found = False
    # Iterate over components to find street
    for component in address_components:
        if 'đường' in component.lower() or 'phố' in component.lower():
            # Remove 'đường' and 'phố' and leading/trailing whitespaces
            street = component.replace('Đường', '').replace('Phố', '').strip()
            streets.append(street)
            street_found = True
            break
    
    if not street_found:
        # If neither 'đường' nor 'phố' is found in any component, append NaN
        streets.append(np.nan)

In [9]:
print("Streets length after =", len(streets))
print("Streets sample after:", streets[0:3])
print("Total number of exceptional cases =", len(street_exception_list))

Streets length after = 28658
Streets sample after: ['Tố Hữu', 'Tố Hữu', 'Đại lộ Thăng Long']
Total number of exceptional cases = 0


In [10]:
streets

['Tố Hữu',
 'Tố Hữu',
 'Đại lộ Thăng Long',
 'Cầu Cốc',
 'Hoàng Hoa Thám',
 'Xuân Đỉnh',
 'Đình Thôn',
 'Từ Hoa',
 'Đại La',
 'Khâm Thiên',
 'Quốc lộ 32',
 'Trần Cung',
 'Trung Kính',
 'Trịnh Văn Bô',
 'Tố Hữu',
 'Phương Canh',
 'Đức Diễn',
 'Thụy Phương',
 'Võ Chí Công',
 'Nguyễn Đình Tứ',
 'Yên Lộ',
 'Ngọc Hà',
 'Tô Hiệu',
 '5',
 'Vũ Hữu',
 'Quốc lộ 32',
 'Phạm Hùng',
 'Nguyễn Đình Thi',
 'Âu Cơ',
 'Ngọc Thụy',
 'Doãn Kế Thiện',
 'Đền Lừ',
 'Kim Giang',
 'Hàng Bạc',
 'Nam Đuống',
 'Phùng Khoang',
 'Phạm Huy Thông',
 'Chiến Thắng',
 nan,
 'Bà Triệu',
 '422B',
 'Nguyên Xá',
 'Nguyễn Chánh',
 'Ngọc Thụy',
 'Kim Đồng',
 'Định Công',
 'Phạm Tuấn Tài',
 'Cổ Nhuế',
 'Nguyễn Xiển',
 'Đại Mỗ',
 'Trung Hòa',
 'Xuân Đỉnh',
 nan,
 'Tố Hữu',
 'Nhật Chiêu',
 'Bằng A',
 nan,
 'Nguyễn Văn Huyên',
 'Lạc Long Quân',
 nan,
 'Lê Trọng Tấn',
 'Kim Ngưu',
 'Trịnh Văn Bô',
 'Nguyễn Xiển',
 'Mễ Trì Hạ',
 'Hoàng Công',
 'Hoàng Quốc Việt',
 nan,
 'Di Trạch',
 'Tây Mỗ',
 'Vĩnh Ninh',
 '70',
 nan,
 'Hàm Nghi',


In [11]:
# Add the "Đường" column to the dataset based on the "streets" list:
dtset.insert(2, "Đường/Phố", streets, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (28658, 18)


,Tiêu đề,Địa chỉ,Đường/Phố,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,186.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,126.9,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,121.2,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự


Add "Phường/Xã" column

In [12]:
ward = [np.nan for i in range(dtset.shape[0])]  # Create a list named "ward" with NaN values, with the length equal to the number of rows in the dataset.
print("Ward length before =", len(ward))  # "ward" is a new column named "Phường" extracted from the "Địa chỉ" column.
print("Ward sample before:", ward[0:3])
#ward_dictionary = {} # Contains exceptional cases that cannot immediately construct the "Phường" column.
ward_exception_list = []  # Contains exceptional cases in a list variable.

Ward length before = 28658
Ward sample before: [nan, nan, nan]


In [13]:
for index in range(dtset.shape[0]):
    if(type(dtset['Địa chỉ'][index]) == float):
        # Skip the current iteration if the value of 'Địa chỉ' at index 'index' is of type float.
        continue
    if 'Phường' not in dtset['Địa chỉ'][index] and 'Xã' not in dtset['Địa chỉ'][index] and 'Thị trấn' not in dtset['Địa chỉ'][index]:
        # If the value of 'Địa chỉ' at index 'index' does NOT contain the strings 'Phường', 'Xã', and 'Thị trấn':
        #ward_dictionary[index] = df['Địa chỉ'][index] # Note this down in ward_dictionary.
        ward_exception_list.append(index)  # Note the exception in ward_exception_list for deletion later.
        continue  # and do nothing.
    address = dtset['Địa chỉ'][index].split(", ")  # Example of an address, then:
    # "Đường Tố Hữu, Phường Dương Nội, Quận Hà Đông, Hà Nội" -> ['Đường Tố Hữu', 'Phường Dương Nội', 'Hà Đông', 'Hà Nội']
    for row in address:
        if 'Phường' in row or 'Xã' in row or 'Thị trấn' in row:
            # If the value of 'Địa chỉ' at index 'index' DOES contain the strings 'Phường', 'Xã', or 'Thị trấn':
            # Use the .strip() method to remove leading and trailing whitespaces.
            new_row = row.replace('Phường', '').replace('Xã', '').replace('Thị trấn', '').strip()
            #new_row = row.strip() # Do not remove 'Phường', 'Xã', or 'Thị trấn' if present.
            if new_row:
                ward[index] = new_row  # Save it as the last result to create a new column.
            else:
                ward[index] = np.nan  # If new_row is empty, assign 'NaN'
            continue
    #print(address)

In [14]:
print("Ward length after =", len(ward))
print("Ward sample after:", ward[0:3])

Ward length after = 28658
Ward sample after: ['Dương Nội', 'La Khê', 'An Thượng']


In [15]:
print("Total number of exceptional cases =", len(ward_exception_list))
#print("ward_dictionary length =", len(ward_dictionary))
#display(ward_dictionary)
print("Example ward_exception_list[0:10]:", ward_exception_list[0:10])
print("Example of exceptional value 1 (in the Dataset):", dtset['Địa chỉ'][ward_exception_list[0]])
print("Example of exceptional value 1 (in the ward variable):", ward[ward_exception_list[0]])

Total number of exceptional cases = 201
Example ward_exception_list[0:10]: [67, 132, 709, 711, 750, 839, 991, 1112, 1126, 1424]
Example of exceptional value 1 (in the Dataset): Dự án Vinhomes Riverside, Quận Long Biên, Hà Nội
Example of exceptional value 1 (in the ward variable): nan


In [16]:
for index in ward_exception_list:
    print(f"Index: {index}, Value in the Dataset: {dtset['Địa chỉ'][index]}")

Index: 67, Value in the Dataset: Dự án Vinhomes Riverside, Quận Long Biên, Hà Nội
Index: 132, Value in the Dataset: Dự án Vinhomes Ocean Park Gia Lâm, Huyện Gia Lâm, Hà Nội
Index: 709, Value in the Dataset: Quan Hoa, Cầu Giấy, Hà Nội.
Index: 711, Value in the Dataset: Dự án Vinhomes Green Bay Mễ Trì, Quận Nam Từ Liêm, Hà Nội
Index: 750, Value in the Dataset: KĐT Phú Lương, Hà Đông, Hà Nội
Index: 839, Value in the Dataset: Khu C Dự án Khu đô thị Geleximco - Lê Trọng Tấn, Quận Hà Đông, Hà Nội
Index: 991, Value in the Dataset: Dự án The Diamond Point, Quận Long Biên, Hà Nội
Index: 1112, Value in the Dataset: Dự án Xanh Villas, Đường Đại lộ Thăng Long, Thạch Thất, Hà Nội
Index: 1126, Value in the Dataset: Dự án An Quý Villa - KĐT Dương Nội, Quận Hà Đông, Hà Nội
Index: 1424, Value in the Dataset: Dự án Vinhomes Green Bay Mễ Trì, Quận Nam Từ Liêm, Hà Nội
Index: 1437, Value in the Dataset: Dự án KĐT Linh Đàm, Hoàng Mai, Hà Nội
Index: 1486, Value in the Dataset: số 10 ngách 40 ngõ 643 Phạm Văn

In [17]:
# Add the "Phường" column to the dataset based on the "ward" variable list:
dtset.insert(3, "Phường/Xã", ward, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (28658, 19)


,Tiêu đề,Địa chỉ,Đường/Phố,Phường/Xã,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Dương Nội,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,186.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,La Khê,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,126.9,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,An Thượng,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,121.2,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự


Add "Quận/Huyện" column

In [18]:
# Create a list named "district" with NaN values, with the length equal to the number of rows in the dataset.
district = [np.nan] * len(dtset)
print("District length before =", len(district))  # "district" is a new column named "Quận" extracted from the "Địa chỉ" column.
print("District sample before:", district[0:3])
dist_exception_list = []  # Contains exceptional cases in a list variable.

District length before = 28658
District sample before: [nan, nan, nan]


In [19]:
# Loop through the dataset
for index in range(len(dtset)):
    # Skip if the value of 'Khu vực' at index 'index' is NaN
    if pd.isnull(dtset['Địa chỉ'][index]):
        continue
    
    # Split address by ", " to extract components
    address_components = dtset['Địa chỉ'][index].split(", ")

    district_found = False
    # Iterate over components to find district
    for component in address_components:
        if 'Quận' in component or 'Huyện' in component:
            # Remove 'Quận' or 'Huyện' and leading/trailing whitespaces
            district_name = component.replace('Quận', '').replace('Huyện', '').strip()
            district[index] = district_name
            district_found = True
            break
    
    if not district_found:
        # If neither 'Quận' nor 'Huyện' is found in any component, consider the last component as district
        if len(address_components) >= 2:
            district[index] = address_components[-2].strip()  # Assuming the district is the second last component
        else:
            dist_exception_list.append(index)  # Note the exception in dist_exception_list for deletion later

In [20]:
print("District length after =", len(district))
print("District sample after:", district[0:3])
print("Total number of exceptional cases =", len(dist_exception_list))

District length after = 28658
District sample after: ['Hà Đông', 'Hà Đông', 'Hoài Đức']
Total number of exceptional cases = 0


In [21]:
for index in dist_exception_list:
    print(f"Index: {index}, Value in the Dataset: {dtset['Địa chỉ'][index]}")

In [22]:
# Add the "Quận" column to the dataset based on the "district" variable list:
dtset.insert(4, "Quận/Huyện", district, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (28658, 20)


,Tiêu đề,Địa chỉ,Đường/Phố,Phường/Xã,Quận/Huyện,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Dương Nội,Hà Đông,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,186.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,La Khê,Hà Đông,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,126.9,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,An Thượng,Hoài Đức,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,121.2,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự


In [23]:
# Count the number of NaN values in column 'Quận/Huyện'
nan_district_count = dtset['Quận/Huyện'].isnull().sum()

# Count the number of NaN values in column 'Phường/Xã'
nan_ward_count = dtset['Phường/Xã'].isnull().sum()

# Count the number of NaN values in column 'Đường/Phố'
nan_street_count = dtset['Đường/Phố'].isnull().sum()

print("Số lượng giá trị NaN trong cột 'Quận/Huyện':", nan_district_count)
print("Số lượng giá trị NaN trong cột 'Phường/Xã':", nan_ward_count)
print("Số lượng giá trị NaN trong cột 'Đường/Phố':", nan_street_count)


Số lượng giá trị NaN trong cột 'Quận/Huyện': 0
Số lượng giá trị NaN trong cột 'Phường/Xã': 201
Số lượng giá trị NaN trong cột 'Đường/Phố': 3280


In [24]:
filtered_rows1 = dtset[dtset['Địa chỉ'].str.contains('kinh doanh tốt')]
filtered_rows1['Địa chỉ']
print("Địa chỉ của dòng số 22196:", dtset['Địa chỉ'][22196])

Địa chỉ của dòng số 22196: Bán nhà, Yên Nghĩa Hà Đông, Xây mới, thiết kế đẹp, thuận tiện giao thông, kinh doanh tốt, gần công viên..


In [25]:
# Set new values for the "Ward/District" column at row number 22196
dtset.at[22196, 'Phường/Xã'] = 'Yên Nghĩa'
dtset.at[22196, 'Quận/Huyện'] = 'Hà Đông'

# Check the values after the changes
print("Giá trị tại cột 'Phường/Xã' tại dòng số 22196:", dtset.at[22196, 'Phường/Xã'])
print("Giá trị tại cột 'Quận/Huyện' tại dòng số 22196:", dtset.at[22196, 'Quận/Huyện'])

Giá trị tại cột 'Phường/Xã' tại dòng số 22196: Yên Nghĩa
Giá trị tại cột 'Quận/Huyện' tại dòng số 22196: Hà Đông


In [26]:
# Check eror columns
contains_HANOI = dtset[dtset['Quận/Huyện'].str.contains('Hà Nội')]
print(contains_HANOI) 
contains_duongngo = dtset[dtset['Quận/Huyện'].str.contains('Đường Ngõ')]
print(contains_duongngo) 
contains_phum = dtset[dtset['Quận/Huyện'].str.contains('Phường Phú m Từ Liêm')]
print(contains_phum)
contains_caudien = dtset[dtset['Quận/Huyện'].str.contains('Phường Cầu Diễn')]
print(contains_caudien)
contains_vangiang = dtset[dtset['Quận/Huyện'].str.contains('Văn Giang')]
print(contains_vangiang) 

                                                 Tiêu đề  \
2806   Bán tòa nhà số 2, ngõ 4 Nguyễn Khuyến, phù hợp...   
15895  Bán tòa nhà số 2, ngõ 4 Nguyễn Khuyến, phù hợp...   

                                                 Địa chỉ            Đường/Phố  \
2806   số 2 Ngõ 4, đường Nguyễn Khuyến, Phường Văn Qu...  đường Nguyễn Khuyến   
15895  số 2 Ngõ 4, đường Nguyễn Khuyến, Phường Văn Qu...  đường Nguyễn Khuyến   

      Phường/Xã Quận/Huyện      Loại tin    Mã tin  \
2806   Văn Quán     Hà Nội    Tin thường  39123583   
15895  Văn Quán     Hà Nội  Tin VIP Bạc  39237097   

                                         Thông tin mô tả  Diện tích Mức giá  \
2806   Bán tòa nhà số 2 Ngõ 4 đường nguyễn Khuyến, Hà...       51.0   243.1   
15895  Bán tòa nhà số 2 Ngõ 4 đường nguyễn Khuyến, Hà...       51.0   243.1   

      Mặt tiền Đường vào Hướng ban công   Hướng nhà  Số tầng  Số phòng ngủ  \
2806     6,5 m     4,5 m     Đông - Nam  Đông - Nam      7.0          12.0   
15895    6,5 m    

In [27]:
# Handle error columns
dtset.loc[2806, 'Quận/Huyện'] = 'Hà Đông'
dtset.loc[15895, 'Quận/Huyện'] = 'Hà Đông'
dtset.loc[11134, 'Quận/Huyện'] = 'Hoàn Kiếm'
dtset.loc[13902, 'Quận/Huyện'] = 'Nam Từ Liêm'
dtset.loc[13902, 'Phường/Xã'] = 'Phú Đô'
dtset.loc[21603, 'Quận/Huyện'] = 'Nam Từ Liêm'
dtset.drop(23040, inplace=True)

### District classification

In [28]:
# Create a new column named "Phân loại quận" with default value NaN
dtset.insert(loc=5, column='Phân loại quận', value=np.nan)

# List of inner districts
inner_districts = ['Ba Đình', 'Hoàn Kiếm', 'Tây Hồ', 'Cầu Giấy', 'Đống Đa', 'Hai Bà Trưng', 'Hoàng Mai', 'Thanh Xuân', 'Long Biên', 'Bắc Từ Liêm', 'Nam Từ Liêm', 'Hà Đông']

# Mark inner districts
dtset.loc[dtset['Quận/Huyện'].isin(inner_districts), 'Phân loại quận'] = 'Nội thành'

# Mark outer districts
dtset.loc[~dtset['Quận/Huyện'].isin(inner_districts), 'Phân loại quận'] = 'Ngoại thành'

# Drop the row with index 2475 (Row had error value of Khu vực)
dtset.drop(index=2475, inplace=True)

dtset.head()

,Tiêu đề,Địa chỉ,Đường/Phố,Phường/Xã,Quận/Huyện,Phân loại quận,Loại tin,Mã tin,Thông tin mô tả,Diện tích,...,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Dương Nội,Hà Đông,Nội thành,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,La Khê,Hà Đông,Nội thành,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,...,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,An Thượng,Hoài Đức,Ngoại thành,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,...,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự
3,"Bán nhà 30m2 x 5 tầng Cầu Cốc, Tây Mỗ, cách cổ...","Phố Cầu Cốc, Phường Tây Mỗ, Nam Từ Liêm, Hà Nội.",Cầu Cốc,Tây Mỗ,Nam Từ Liêm,Nội thành,Tin thường,39214622,"Bán nhà 30m² x 5 tầng Cầu Cốc, Tây Mỗ, cách cổ...",30.0,...,"3,54 m",2 m,Tây,Tây,5.0,3.0,4.0,Sổ đỏ/ Sổ hồng.,Đầy đủ,Nhà riêng
4,"Bán biệt thự liền kề Ba Đình, nội thất dát vàn...","Dự án 671 Hoàng Hoa Thám, Đường Hoàng Hoa Thám...",Hoàng Hoa Thám,Liễu Giai,Ba Đình,Nội thành,Tin thường,38856881,Bán căn biệt thự liền kề Viglacera Ba Đình. Hà...,90.0,...,"5,2 m",10 m,NaN,NaN,5.0,6.0,6.0,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự


### Handle "Mặt tiền", "Đường vào" columns

In [29]:
dtset['Mặt tiền'] = dtset['Mặt tiền'].str.replace(' m', '').str.replace(',', '.')
dtset['Mặt tiền'] = dtset['Mặt tiền'].astype(float)
dtset['Đường vào'] = dtset['Đường vào'].str.replace(' m', '').str.replace(',', '.')
dtset['Đường vào'] = dtset['Đường vào'].astype(float)

In [30]:
dtset.head(3)

,Tiêu đề,Địa chỉ,Đường/Phố,Phường/Xã,Quận/Huyện,Phân loại quận,Loại tin,Mã tin,Thông tin mô tả,Diện tích,...,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Dương Nội,Hà Đông,Nội thành,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,La Khê,Hà Đông,Nội thành,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,...,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,An Thượng,Hoài Đức,Ngoại thành,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,...,5.0,15.0,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự


### Handle "Thông tin mô tả" column

In [31]:
# Select rows related to schools
school_data = dtset[
    (dtset['Thông tin mô tả'].str.contains('trường|học viện', case=False, na=False)) &
    ~((dtset['Thông tin mô tả'].str.contains('thị trường|môi trường|Trường Chinh|Nguyễn Trường Tộ|Phan Văn Trường|Mr Trường|Mr. Trường|Liên hệ Trường|Trường Thổ Thổ|Trường Triệu Đô|trường tồn|quảng trường|Trường Lâm|trường xuân|em Trường|E Trường|Xuân Trường|: Trường|anh Trường|Trường_Chinh|Trường Sa|Trường Yên|trường phái|trường hợp', case=False, na=False)) &
    ~(dtset['Thông tin mô tả'].str.contains('trường tư|nhà trường|tốt nhất|ngay trường|đào tạo|Nhân Văn|danh tiếng|giáo dục|quốc gia|lien cấp|trường Mai Động|trường Ngô|trường Pháp|Phenika|Kinh doanh|cạnh trường|trường đoàn|trường, |cấp ba|trường Kim Liên|trường 1|nhìn trường|trường Lý|trường Thực Nghiệm|trường, trạm|tiện trường|trường nổi tiếng|Tài Chính|trường Đảng|trường Unis|C3|trường Trí Đức|liền cấp|Well|trung học|trường Bill|đến trường|trường Kinh tế|học viện|Sát trường|trường Ngoại Giao|Trường FPT|trường - trạm|trường Lo|trường tiểu|Trường. Chợ|nhiều trường|trường Ái Mộ|trường Kiến Trúc|trường điểm|trường Hoang|trường khọc|trường Ngô Sỹ Liên|trường Lương Thế Vinh|trường Cao Đẳng|trường Nông Nghiệp|sân trường|trường Ban Mai|trường Nguyễn|trường Bách|Thương Mại|trường Thái Thịnh|Academy|DH|trường trạm|Việt Pháp|trường C1|xung quanh trường|trường chuyên|Vinuni|trường mần non|trường cán bộ quản lý|trường học|trường công|đại học|ĐH|liên cấp|mầm non|trường Lê|gần trường|gần trường|các cấp|THPT|THCS|mẫu giáo|tiểu học|trường quốc tế|trường An Ninh|hệ thống giáo dục|cấp 2|cấp 1|cấp 3|hệ thống trường|gần các trường|trường Nguyễn Siêu|Chu Văn An|Wellsping|Vinschool|trường chợ|chợ trường|trường Gia|trường, chợ|chợ, trường|tiện ích trường|trường - chợ|Trường Thanh Liệt', case=False, na=False)))]

# Add a binary column indicating presence of school
dtset['Nhóm 1-Trường học'] = 0
dtset.loc[school_data.index, 'Nhóm 1-Trường học'] = 1

# Select rows related to markets
market_data = dtset[dtset['Thông tin mô tả'].str.contains('chợ|siêu thị|trung tâm thương mại', case=False, na=False)]
# Add a binary column indicating presence of market
dtset['Nhóm 1- Chợ/Siêu thị'] = 0
dtset.loc[market_data.index, 'Nhóm 1- Chợ/Siêu thị'] = 1

# Select rows related to hospitals
hospital_data = dtset[dtset['Thông tin mô tả'].str.contains('bệnh viện|trung tâm chăm sóc|Vinmec', case=False, na=False)]
# Add a binary column indicating presence of hospital
dtset['Nhóm 1-Bệnh viện'] = 0
dtset.loc[hospital_data.index, 'Nhóm 1-Bệnh viện'] = 1

# Select rows related to playgrounds
playground_data = dtset[dtset['Thông tin mô tả'].str.contains('khu vui chơi|công viên|trẻ em|vườn hoa', case=False, na=False)]
# Add a binary column indicating presence of playground
dtset['Công viên'] = 0
dtset.loc[playground_data.index, 'Công viên'] = 1

# Select rows related to narrow alleys
narrow_alley_data = dtset['Thông tin mô tả'].str.contains('tránh|2 chiều|chánh|song song', case=False, na=False) & \
                     ~dtset['Thông tin mô tả'].str.contains('ba gác|thời gian|3 gác|xe máy|tranh chấp|bac gác|làm phiền|mất giá|ba ngác|từ thiện|tránh xa|miễn cưỡng|SH|Bagac|lạm phát|tránh dịch|mất lượt|ảnh|5 tránh|quy', case=False, na=False)
wide_alley_data = dtset['Thông tin mô tả'].str.contains('ba gác|3 gác|xe máy|bac gác|ba ngác|Bagac| SH ', case=False, na=False)

# Add a column indicating alley type
dtset['Loại ngõ'] = None
dtset.loc[narrow_alley_data, 'Loại ngõ'] = 'Nhỏ'
dtset.loc[wide_alley_data, 'Loại ngõ'] = 'Rộng'

# Select rows related to pathways
pathway_data = dtset['Thông tin mô tả'].str.contains('vỉa hè', case=False, na=False)
# Add a binary column indicating presence of pathway
dtset['Vỉa hè'] = 0
dtset.loc[pathway_data, 'Vỉa hè'] = 1

# Select rows related to urgent sales
urgent_sale_data = dtset[
    (dtset['Thông tin mô tả'].str.contains('gấp|cắt lỗ|bán nhanh', case=False, na=False)) &
    ~((dtset['Thông tin mô tả'].str.contains('gấp đôi|Gọi gấp|Xem nhà gấp|gấp 2|gấp bội|đắt gấp|gấp rút hoàn thành|Liên hệ gấp|gấp rút|Lh gấp|Em Trang Gấp|tăng giá|LH|mua gấp|tăng gấp|xem gấp|Gấp.', case=False, na=False)) &
    ~(dtset['Thông tin mô tả'].str.contains('vốn xoay|giao dịch|bay gấp|biệt thự|Gấp hạ|Gấp,|chủ gấp|bán nhanh|sang gấp|gấp bán|cắt lỗ|thu tiền|gấp trong|xử lý|gấp anh|bán nhà gấp|Gấp -|cắt lỗ gấp|ban gấp|gấp nhà|chốt gấp|bán ngay|việc gấp|muốn gấp|nhượng lại|Gấp gấp|bấn gấp|bân gấp|cần gấp|dòng tiền|gấp chỉ|cần tiền gấp|cực gấp|bán lại|lo việc gấp|thanh khoản gấp|Giá đẩy gấp|rất gấp|nhượng gấp|thanh lý gấp|ra gấp|chuyển đổi gấp|gấp lắm|bán gấp', case=False, na=False)))
]

# Add a binary column indicating urgent sale
dtset['Bán gấp'] = 0
dtset.loc[urgent_sale_data.index, 'Bán gấp'] = 1

# Select rows related to rooftop spaces
rooftop_data = dtset['Thông tin mô tả'].str.contains('sân phơi', case=False, na=False)
# Add a binary column indicating presence of rooftop space
dtset['Sân phơi'] = 0
dtset.loc[rooftop_data, 'Sân phơi'] = 1

# Select rows related to airy spaces
airiness_data = dtset['Thông tin mô tả'].str.contains('thoáng', case=False, na=False)
# Add a binary column indicating airy spaces
dtset['Nhà thoáng'] = 0
dtset.loc[airiness_data, 'Nhà thoáng'] = 1

# Select rows related to proximity to lakes
lake_data = dtset['Thông tin mô tả'].str.contains(' hồ ', case=False, na=False)
# Add a binary column indicating proximity to lakes
dtset['Gần hồ'] = 0
dtset.loc[lake_data, 'Gần hồ'] = 1

# Select rows related to safety
safety_data = dtset[
    (dtset['Thông tin mô tả'].str.contains('an ninh|an sinh|dân trí|văn minh', case=False, na=False)) &
    ~((dtset['Thông tin mô tả'].str.contains('học viện an ninh|trường an ninh|đại học An Ninh|ĐH An Ninh', case=False, na=False)) &
    ~(dtset['Thông tin mô tả'].str.contains('an sinh|dân trí', case=False, na=False))
)
]
# Add a binary column indicating safety
dtset['An ninh'] = 0
dtset.loc[safety_data.index, 'An ninh'] = 1

# Select rows related to modern features
modern_features_data = dtset['Thông tin mô tả'].str.contains('hiện đại|sang trọng|xịn xò|xịn sò|sịn xò|cao cấp', case=False, na=False)
# Add a binary column indicating modern features
dtset['Hiện đại'] = 0
dtset.loc[modern_features_data, 'Hiện đại'] = 1

# Select rows related to garage facilities
garage_data = dtset['Thông tin mô tả'].str.contains('gara|đỗ cửa| hầm |ga ra', case=False, na=False)
# Add a binary column indicating garage facilities
dtset['Chỗ để xe'] = 0
dtset.loc[garage_data, 'Chỗ để xe'] = 1

# Select rows related to elevator availability
elevator_data = dtset['Thông tin mô tả'].str.contains('thang máy', case=False, na=False)
# Add a binary column indicating elevator availability
dtset['Thang máy'] = 0
dtset.loc[elevator_data, 'Thang máy'] = 1

# Select rows related to corner locations
corner_data = dtset['Thông tin mô tả'].str.contains('lô góc|căn góc', case=False, na=False)
# Add a binary column indicating corner locations
dtset['Lô góc'] = 0
dtset.loc[corner_data, 'Lô góc'] = 1

# Select rows related to properties offering financial support
support_data = dtset['Thông tin mô tả'].str.contains('hỗ trợ', case=False, na=False) & \
               ~dtset['Thông tin mô tả'].str.contains('không hỗ trợ', case=False, na=False)
# Add a binary column indicating financial support
dtset['Hỗ trợ vay'] = 0
dtset.loc[support_data, 'Hỗ trợ vay'] = 1

# Select rows related to mini apartments
mini_apartment_data = dtset['Thông tin mô tả'].str.contains('CCMN|mini|mi ni', case=False, na=False)
# Add a column indicating apartment type
dtset.loc[mini_apartment_data, 'Loại hình'] = 'Chung cư mini'

In [32]:
dtset.head(5)

,Tiêu đề,Địa chỉ,Đường/Phố,Phường/Xã,Quận/Huyện,Phân loại quận,Loại tin,Mã tin,Thông tin mô tả,Diện tích,...,Bán gấp,Sân phơi,Nhà thoáng,Gần hồ,An ninh,Hiện đại,Chỗ để xe,Thang máy,Lô góc,Hỗ trợ vay
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",Tố Hữu,Dương Nội,Hà Đông,Nội thành,Tin thường,39159568,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,...,1,0,0,0,1,0,0,0,0,0
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",Tố Hữu,La Khê,Hà Đông,Nội thành,Tin thường,38451107,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,...,0,0,1,0,0,1,1,0,0,0
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",Đại lộ Thăng Long,An Thượng,Hoài Đức,Ngoại thành,Tin thường,39204669,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,...,0,0,0,0,1,0,0,0,1,0
3,"Bán nhà 30m2 x 5 tầng Cầu Cốc, Tây Mỗ, cách cổ...","Phố Cầu Cốc, Phường Tây Mỗ, Nam Từ Liêm, Hà Nội.",Cầu Cốc,Tây Mỗ,Nam Từ Liêm,Nội thành,Tin thường,39214622,"Bán nhà 30m² x 5 tầng Cầu Cốc, Tây Mỗ, cách cổ...",30.0,...,0,0,0,0,0,0,0,0,0,0
4,"Bán biệt thự liền kề Ba Đình, nội thất dát vàn...","Dự án 671 Hoàng Hoa Thám, Đường Hoàng Hoa Thám...",Hoàng Hoa Thám,Liễu Giai,Ba Đình,Nội thành,Tin thường,38856881,Bán căn biệt thự liền kề Viglacera Ba Đình. Hà...,90.0,...,0,0,0,0,1,1,0,1,0,0


### Drop unvalued columns

In [33]:
# Print the initial dataset size
print("Initial dataset size:", dtset.shape)

# Drop specified columns
dtset.drop(['Địa chỉ', 'Tiêu đề', 'Thông tin mô tả'], axis=1, inplace=True)

# Print the dataset size after removing some attribute columns
print("Dataset size after removing some attribute columns:", dtset.shape)

Initial dataset size: (28656, 37)
Dataset size after removing some attribute columns: (28656, 34)


In [34]:
# Update the list of columns after removing some columns
dtset_columns_after = dtset.columns.tolist()

In [35]:
# Summary
# Initialize an empty list to store removed columns
delete_col_count = []

# Iterate through the original list of columns
for element in dtset_columns:
    # Check if the column is not in the updated list of columns
    if element not in dtset_columns_after:
        # If the column is not in the updated list, append it to the list of removed columns
        delete_col_count.append(element)

# Print the removed columns and their count
print("The removed attribute columns:", delete_col_count)
print("Total number of removed columns:", len(delete_col_count))

The removed attribute columns: ['Tiêu đề', 'Địa chỉ', 'Ngày đăng', 'Ngày hết hạn', 'Người đăng', 'Thông tin mô tả']
Total number of removed columns: 6


In [36]:
dtset.shape

(28656, 34)

In [37]:
# Save the DataFrame to a CSV file 
dtset.to_csv('nhaban_eng.csv', index=False)

In [38]:
# Save the "Thỏa thuận" DataFrame to a CSV file 
new_file = dtset[dtset['Mức giá'] == 'Thỏa thuận']
new_file.to_csv('thoa_thuan_nhaban.csv', index=False)

In [39]:
# Save the remaining DataFrame to a CSV file 
remaining_file = dtset[dtset['Mức giá'] != 'Thỏa thuận']
# remaining_file['Mức giá'] = pd.to_numeric(remaining_file['Mức giá'], errors='coerce')
remaining_file.to_csv('remaining_nhaban.csv', index=False)